In [ ]:
!pip3 install scikit-image --user

In [1]:
import os
import png
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

from skimage import transform
from skimage.io import imread
from skimage.color import rgb2gray
from sklearn.metrics import classification_report, confusion_matrix

Using TensorFlow backend.


In [7]:
"""
    Preprocess image, normalizing and resizing it

    :param frame: RGBA frame
"""    
def preprocess_image(frame):
    
    # Normalize Pixel Values
    normalized_frame = frame/255.0 - 0.5
    
    # Resize
    preprocessed_frame = transform.resize(normalized_frame, [150,150])
    
    return preprocessed_frame

def make_labels(labels):
    np_labels = np.zeros((len(labels), 16))
    
    for i in range(len(labels)):
        np_labels[i, labels[i]] = 1
    
    return np_labels

In [8]:
model = Sequential()
model.add(Conv2D(20, kernel_size=(7, 7), strides=(1, 1),
                 activation='relu',
                 input_shape=(150, 150, 1)))
model.add(MaxPooling2D(pool_size=(4, 4), strides=(1, 1)))
model.add(Conv2D(50, kernel_size = (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(16, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01),
              metrics=['accuracy'])

In [ ]:
train_labels = []
val_labels = []
test_labels = []

train_input = []
val_input = []
test_input = []

folder = "./data/rvl-cdip"
labels_file = open(folder + "/labels/val.txt")

for line in labels_file:
    sp = line.split()
    pp_img = preprocess_image(imread(folder + "/images/" + sp[0]))
    
    val_input.append(pp_img)
    val_labels.append(int(sp[1]))
    
    if len(val_input) >= 100:
        break

x_val = np.expand_dims(np.asarray(val_input), axis=3)
y_val = make_labels(val_labels)

labels_file = open(folder + "/labels/train.txt")

for line in labels_file:
    sp = line.split()
    pp_img = preprocess_image(imread(folder + "/images/" + sp[0]))
    
    train_input.append(pp_img)
    train_labels.append(int(sp[1]))
    
    if len(train_input) >= 100:
        x_train = np.expand_dims(np.asarray(train_input), axis=3)
        y_train = make_labels(train_labels)
        
        
        model.fit(x_train, y_train,
                  batch_size=10,
                  epochs=1,
                  verbose=1,
                  validation_data=(x_val, y_val))
        
        train_input = []
        train_labels = []
        
#     path = sp[0].split('/')
#     pp_path = folder + "/images_pp/" + "/".join(path[:-1])
#     if not os.path.exists(pp_path):
#         os.makedirs(pp_path)
#     pp_path = pp_path + '/' + str(path[-1].split('.')[0]) + "_pp"
#     np.save(pp_path, pp_img)
    
#     train_path.append(pp_path)
#     train_labels.append(sp[1])   

Train on 100 samples, validate on 100 samples
Epoch 1/1
100/100 [==============================] - 33s 326ms/step - loss: 2.6485 - acc: 0.1700 - val_loss: 2.7781 - val_acc: 0.1100
Train on 100 samples, validate on 100 samples
Epoch 1/1
100/100 [==============================] - 32s 319ms/step - loss: 2.7137 - acc: 0.1000 - val_loss: 2.7540 - val_acc: 0.1500
Train on 100 samples, validate on 100 samples
Epoch 1/1
100/100 [==============================] - 32s 320ms/step - loss: 2.7562 - acc: 0.0700 - val_loss: 2.7394 - val_acc: 0.1500
Train on 100 samples, validate on 100 samples
Epoch 1/1
100/100 [==============================] - 30s 297ms/step - loss: 2.7293 - acc: 0.1300 - val_loss: 2.7207 - val_acc: 0.1600
Train on 100 samples, validate on 100 samples
Epoch 1/1
100/100 [==============================] - 28s 280ms/step - loss: 2.7233 - acc: 0.1100 - val_loss: 2.7080 - val_acc: 0.1700
Train on 100 samples, validate on 100 samples
Epoch 1/1
100/100 [==============================] - 2

In [ ]:
np_img_type = np.zeros((len(img_type), dt))

print(len(img_type))

for i in range(len(img_type)):
    np_img_type[i, img_type[i]] = 1
print(np_img_type)

In [ ]:
x_train = np.expand_dims(np.asarray(imgs[:2000]), axis=3)
y_train = np_img_type[:2000,:]

x_val = np.expand_dims(np.asarray(imgs[2000:4000]), axis=3)
y_val = np_img_type[2000:4000,:]

x_test = np.expand_dims(np.asarray(imgs[4000:]), axis=3)
y_test = np_img_type[4000:,:]

print(y_train.shape)

In [ ]:
model.fit(x_train, y_train,
          batch_size=100,
          epochs=2,
          verbose=1,
          validation_data=(x_val, y_val))

In [ ]:
#Confution Matrix and Classification Report
Y_pred = model.predict_classes(x_test)

In [ ]:
print(Y_pred)

In [ ]:
y_pred = Y_pred
y_true = np.argmax(y_test, axis=1)
print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred))